<a href="https://colab.research.google.com/github/cechieh-code/vaccine/blob/main/mapping_of_research_vaccine_production_capacity_in_Africa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#An Assessment of Capacity of Vaccine Production in Africa
Created: 02/10/2023

In [ ]:
import matplotlib.pylab as plt
import numpy as np 
import seaborn as sns
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor

from sklearn.linear_model import RANSACRegressor
%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving biblio_no_coma_csv.csv to biblio_no_coma_csv.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['biblio_no_coma_csv.csv']))
names = ("County" , "GDP (million)" "Population(thousand)" "Pubs" "Cit" "F-a pubs" "F-a cit" "2008 pubs" "2017 pubs" "Int Coll" "Nat Coll" "Inst Coll" "Single author")
# Bibliometric Dataset is now stored in a Pandas Dataframe as df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               54 non-null     object 
 1   GDP (million)         54 non-null     float64
 2   Population(thousand)  54 non-null     float64
 3   Pubs                  54 non-null     float64
 4   Cit                   54 non-null     float64
 5   F-a pubs.             54 non-null     float64
 6   F-a cit.              54 non-null     float64
 7   2008 pubs.            54 non-null     float64
 8   2017 pubs.            54 non-null     float64
 9   Int Coll.             54 non-null     float64
 10  Nat Coll.             54 non-null     float64
 11  Inst Coll.            54 non-null     float64
 12  Single author         54 non-null     float64
dtypes: float64(12), object(1)
memory usage: 5.6+ KB


In [ ]:
print(df)

                             Country  GDP (million)  Population(thousand)  \
0                            Algeria       159049.0               40606.0   
1                             Angola        95335.0               28813.0   
2                              Benin         8583.0               10872.0   
3                           Botswana        15581.0                2250.0   
4                       Burkina Faso        11693.0               18646.0   
5                            Burundi         3007.0               10524.0   
6                           Cameroon        32218.0               23439.0   
7                         Cape Verde         1617.0                 540.0   
8           Central African Republic         1756.0                4595.0   
9                               Chad         9601.0               14453.0   
10                           Comoros          617.0                 796.0   
11  Democratic Republic of the Congo        35382.0               78736.0   

In [ ]:
AMU = "Algeria" "Libya" "Mauritania" "Morocco" "Tunisia"
CEN_SAD = "Benin" "Burkina_Faso" "Cape_Verde" "CAR" "Chad" "Comoros" "Djibuti" "Egypt" "Eritrea" "Gambia" "Ghana" "Guinea_Bissau" "Ivory_coast" "Kenya" "Liberia" "Libya" "Mali" "Mauritania" "Morocco" "Niger" "Nigeria" "SaoTome_Principe" "Senegal" "Sierra_leone" "Somalia" "Togo" "Tunisia"
COMESA = "Burundi" "Comoros" "DR_Congo" "Djibuti" "Egypt" "Eritrea" "Eswatini" "Ethiopia" "Kenya" "Libya" "Madagascar" "Malawi" "Mauritius" "Rwanda" "Seychelles" "Somalia"  "Sudan" "Tunisia" "Uganda" "Zambia" "Zimbabwe" 
EAC = "Congo" "Burundi" "Kenya" "Rwanda" "South_Sudan" "Uganda" "Tanzania"
ECCAS = "Gabon" "DR_Congo" "Cameroun" "Rwanda" "Eq_Guinea" "SaoTome_Principe" "CAR" "Chad" "Angola" "Burundi" 
ECOWAS = "Cameroun" "Bukina_Faso" "Senegal" "Togo" "Nigeria" "Mali" "Ghana" "Benin" "Guinea" "Gambia"
IGAD = "Djibuti" "Eritrea" "Ethiopia" "Kenya" "Somalia" "South_Sudan" "Sudan" "Uganda" 
SADC = "South_Africa" "Mozambique" "Zimbabwe" "Mauritius" "Seychelles" "Namibia" "Comoro" "Zambia"

In [ ]:
# summary statistics for all numerical columns
round(df.describe(),2)

,GDP (million),Population(thousand),Pubs,Cit,F-a pubs.,F-a cit.,2008 pubs.,2017 pubs.,Int Coll.,Nat Coll.,Inst Coll.,Single author
count,54.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00,54.00
mean,40628.46,22647.13,5513.65,36686.31,1949.56,11692.78,332.44,734.11,80.22,2.53,15.00,2.05
std,81419.76,32328.76,12475.96,53636.30,4425.02,23629.88,743.71,1633.60,15.40,3.84,11.79,1.77
min,343.00,95.00,25.00,0.00,0.00,0.00,0.00,7.00,36.30,0.00,0.00,0.00
25%,3902.75,2935.25,333.00,2869.00,64.75,419.50,15.75,45.00,75.45,0.20,6.58,0.75
50%,10816.50,12313.50,1516.00,17214.50,432.00,3354.50,79.00,193.50,83.75,0.90,12.30,1.75
75%,35211.25,27379.00,3575.50,44107.50,1332.00,9368.75,232.50,472.00,90.72,2.80,19.55,2.62
max,404653.00,185990.00,63171.00,243026.00,25853.00,117263.00,4210.00,8325.00,100.00,14.20,47.30,7.40


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Clinical trial infrastructures_mapping paper.csv to Clinical trial infrastructures_mapping paper.csv


In [ ]:
#Store Clinical Trial Infrastructure in dataframe as df2
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Clinical trial infrastructures_mapping paper.csv']))

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               54 non-null     object 
 1   GDP (million)         54 non-null     float64
 2   Pop. (thousand)       54 non-null     float64
 3   R&D staff per mill    54 non-null     float64
 4   Researchers per mill  54 non-null     float64
 5   Researchers (% M&HS   53 non-null     float64
 6   Researchers (%PhD)    54 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.1+ KB


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving R and D expenditure_map_float.csv to R and D expenditure_map_float (1).csv


In [ ]:
#R and D expenditure in dataframe as df3
import io
df3 = pd.read_csv(io.BytesIO(uploaded['R and D expenditure_map_float.csv']))

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              54 non-null     object 
 1   GDP (million)        54 non-null     float64
 2   Pop (thousand)       54 non-null     float64
 3   GERD                 54 non-null     float64
 4   GERD (% GDP)         54 non-null     float64
 5   GERD per capita      54 non-null     float64
 6   GERD (M&HS)          54 non-null     float64
 7   GERD per researcher  54 non-null     float64
 8   GERD (M&HS %)        54 non-null     float64
dtypes: float64(8), object(1)
memory usage: 3.9+ KB


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined dataset_map.csv to combined dataset_map.csv


In [ ]:
#Store combined dataset of df, df2, and df3 as df4
import io
df4 = pd.read_csv(io.BytesIO(uploaded['combined dataset_map.csv']))

In [ ]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               54 non-null     object 
 1   GDP (million)         54 non-null     float64
 2   Population(thousand)  54 non-null     float64
 3   Pubs                  54 non-null     float64
 4   Cit                   54 non-null     float64
 5   F-a pubs.             54 non-null     float64
 6   F-a cit.              54 non-null     float64
 7   2008 pubs.            54 non-null     float64
 8   2017 pubs.            54 non-null     float64
 9   Int Coll.             54 non-null     float64
 10  Nat Coll.             54 non-null     float64
 11  Inst Coll.            54 non-null     float64
 12  Single author         54 non-null     float64
 13  R&D staff per mill    54 non-null     float64
 14  Researchers per mill  54 non-null     float64
 15  Researchers (% M&HS   53 